Hola &#x1F600;

Soy **Hesus Garcia**  como "Jesús" pero con H. Sé que puede ser confuso al principio, pero una vez que lo recuerdes, ¡nunca lo olvidarás! &#x1F31D;	. Como revisor de código de Triple-Ten, estoy emocionado de examinar tus proyectos y ayudarte a mejorar tus habilidades en programación. si has cometido algún error, no te preocupes, pues ¡estoy aquí para ayudarte a corregirlo y hacer que tu código brille! &#x1F31F;. Si encuentro algún detalle en tu código, te lo señalaré para que lo corrijas, ya que mi objetivo es ayudarte a prepararte para un ambiente de trabajo real, donde el líder de tu equipo actuaría de la misma manera. Si no puedes solucionar el problema, te proporcionaré más información en la próxima oportunidad. Cuando encuentres un comentario,  **por favor, no los muevas, no los modifiques ni los borres**. 

Revisaré cuidadosamente todas las implementaciones que has realizado para cumplir con los requisitos y te proporcionaré mis comentarios de la siguiente manera:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

</br>

**¡Empecemos!**  &#x1F680;

# Introducción

Soy empleado de una empresa emergente de alimentos y necesito estudiar el comportamiento de los usuarios de la aplicación de la empresa.

En primer lugar, necesito estudiar el embudo de ventas. Quiero saber cómo llegan los usuarios a la etapa de compra. ¿Cuántos usuarios llegan realmente a esta etapa? ¿Cuántos se quedaron atascados en los pasos anteriores?

Luego analizaré los resultados de la prueba A/A/B. Los diseñadores quieren cambiar las fuentes de toda la aplicación, pero los gerentes temen que el nuevo diseño pueda intimidar a los usuarios. Los usuarios se dividen en tres grupos: dos grupos de control reciben fuentes antiguas y un grupo de prueba recibe fuentes nuevas. Vamos a averiguar qué conjunto de fuentes proporciona la mejor conversión.

# Descripción de los datos
Cada entrada de registro es una acción de usuario o un evento:
 - EventName: nombre del evento.
 - DeviceIDHash: identificador de usuario unívoco.
 - EventTimestamp: hora del evento.
 - ExpId: número de experimento: 246 y 247 son los grupos de control, 248 es el grupo de prueba.

## Descarga de datos y preparación para el análisis.

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import math
from scipy import stats as st
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from plotly import graph_objects as go

pd.options.display.float_format = "{:.2f}".format
color_dict = ['#b83800','#00b838','#4800eb','#ffcf05','#5f3600','#d938ff','#004c3d']

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Tu código no tiene ningun error, pero quería proporcionarte algunos comentarios sobre la organización de los imports en tu código. Entiendo que esto se te proporcionó como parte de una plantilla, sin embargo es importante destacar el orden de los imports. 
    
Es preferible agrupar los imports siguiendo el siguiente orden:

Imports de la biblioteca estándar de Python.
Imports de bibliotecas de terceros relacionadas.
Imports específicos de la aplicación local o biblioteca personalizada.
Para mejorar la legibilidad del código, también es recomendable dejar una línea en blanco entre cada grupo de imports, pero solo un import por línea.
Te dejo esta referencia con ejemplos:  
https://pep8.org/#imports

</div>

Análisis preliminar de la estructura de datos.

In [ ]:
logs_path   = 'logs_exp_us.csv'
platform_path = 'https://code.s3.yandex.net/datasets/'

try:
    logs   = pd.read_csv(logs_path, sep='\t', nrows=500)
except:
    logs  = pd.read_csv(platform_path+logs_path, sep='\t', nrows=500)

In [ ]:
logs.info()

In [ ]:
logs[:5]

In [ ]:
logs['EventName'].value_counts()

Carga de datos en forma optimizada.

In [ ]:
try:
    logs = pd.read_csv(logs_path, sep='\t', 
                             dtype={'EventName': 'category'})
except:
    logs = pd.read_csv(platform_path+logs_path, sep='\t', 
                             dtype={'EventName': 'category'})

## Preprocesamiento de datos

In [ ]:
logs.info()

No se encontraron datos faltantes. Cambiaré el nombre de las columnas.

In [ ]:
logs.columns

In [ ]:
logs.columns = ('event','user_id','timestamp','exp_id')

- Agregaré una columna de fecha y hora y una columna separada para fechas.

In [ ]:
logs.isnull().sum()

In [ ]:
logs['datetime'] = logs['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
logs['date'] = logs['datetime'].astype('datetime64[D]')
logs[:5]

Voy a encontrar duplicados y valores faltantes.

In [ ]:
print('Existen {:} duplicados en la mesa. Esto es el {:.2%} de toda la data.'.
      format(logs.duplicated().sum(),logs.duplicated().sum()/len(logs)))

In [ ]:
logs.drop_duplicates(keep= 'last', inplace=True)
print('Duplicados encontrados - ',logs.duplicated().sum(),' records')

In [ ]:
users_wrong = logs.groupby('user_id', as_index = False).agg({'exp_id':'nunique'})
print('{:} usuarios que han participado en más de un experimento'.
      format(len(users_wrong.query('exp_id > 1'))))

- Conclusión

En conclusión, todo salió bien durante el proceso de limpieza de datos. No se encontraron valores faltantes, se identificaron y eliminaron duplicados, se simplificaron los nombres de las columnas para una mejor comprensión y se convirtió el campo "EventName" en una variable categórica.

<div class="alert alert-block alert-success">

<b>Comentario del revisor</b> <a class="tocSkip"></a>

Muy bien, hiciste un conciso análisis exploratorio de datos pero pertinente dado el dataset otorgado, a continuación algunos comentarios:
    
<ul>
    <li>Excelente en la carga y exploración inicial de datos con métodos tradicionales, la parte de los duplicados está también bastante bien hecha, esto a veces se puede deber a errores de los logs o un usuario realizando una acción bastante rápido que el sistema no alcanza a detectar o duplica.</li>
    <li>Muy bien el procesamiento, de forma rápida y concisa creaste el dataframe con los datos esperados, separando la información de los logs y usando una columna de fecha y una columna de fecha y hora.</li>
    <li>Muy bien por el uso adecuado de tipo de datos. Sería de mucha ayuda tener un info final después de haber finalizado todo tu procesamiento para mayor claridad inmediata.</li>
</ul>
</div>

## Estudiar y comprobar los datos
### ¿Cuántos eventos hay en los registros?

In [ ]:
event_number = logs.groupby('exp_id', as_index = False).agg({'event':'count'})
event_number

In [ ]:
def plot_bar_for_events(df, 
                        x_column, 
                        y_column, 
                        tickvals2, 
                        texttemplate = '%{text:.4s}',
                        title = "Title",
                        xaxis_title = "Número de prueba",
                        yaxis_title = 'Date'):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df[x_column],
                         y=df[y_column],
                         text=df[y_column],
                         texttemplate=texttemplate,
                         textposition='outside',
                         textfont_size=14,
                         marker=dict(
                             color='rgba(196, 207, 200 0.6)',
                             line=dict(
                                 color='rgba(18, 63, 36,  1.0)',
                                 width=1),
                         )
                        ))
    fig.update_layout(
        title=title,
        xaxis_tickfont_size=16,
        yaxis=dict(
            title=yaxis_title,
            titlefont_size=18,
            tickfont_size=16,
        ),
        xaxis=dict(
            title=xaxis_title,
            titlefont_size=18,
            tickfont_size=16,
            tickmode = 'array',
            tickvals = tickvals
        ),
        bargroupgap=0.1
    )
    
    fig.show()

In [ ]:
tickvals = list(event_number['exp_id'])
plot_bar_for_events(event_number,'exp_id', 'event', tickvals, 
                    title = "Número de eventos en cada prueba",
                    yaxis_title = 'Número de eventos' )

In [ ]:
print('Existen {:} eventos en la base de datos'.format(event_number['event'].sum()))

### ¿Cuántos usuarios y usuarias hay en los registros?

In [ ]:
users_number = logs.groupby('exp_id', as_index = False).agg({'user_id':'nunique'})
users_number

In [ ]:
tickvals = list(users_number['exp_id'])
plot_bar_for_events(users_number,'exp_id', 'user_id', tickvals,
                    texttemplate='%{text:.}',
                    title = "Número de usuarios en cada prueba",
                    yaxis_title = 'Número de usuarios' )

In [ ]:
print('Existen {:} usuarios en la base de datos'.format(users_number['user_id'].sum()))

In [ ]:
print('El número de usuarios únicos es {:}'.format(logs['user_id'].nunique()))

### ¿Cuál es el promedio de eventos por usuario?
#### Todos los eventos

In [ ]:
users_events = (
    logs.groupby('user_id', as_index = False).agg({'event':'count'})
)
users_events = users_events.rename(columns = {'event':'count_events'})
users_events[:4]

- Distribución del número de eventos por usuario.

In [ ]:
fig = px.scatter(users_events, x='user_id', y='count_events')
fig.update_layout(
        title='Distribución del número de eventos por usuario.',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Número de eventos',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(
            title='Usuarios',
            titlefont_size=16,
            tickfont_size=14,
            tickmode = 'array',
            tickvals = []
        ))
fig.show()

Existen valores anormalmente grandes.

In [ ]:
percentile_dict = [95, 99]
percentile = np.percentile(users_events['count_events'], percentile_dict)
print('No más del {:}% de los usuarios tuvieron más de {:.0f} eventos'.format(100 - percentile_dict[0],percentile[0]))
print('No más del {:}% de los usuarios tuvieron más de {:.0f} eventos'.format(100 - percentile_dict[1],percentile[1]))

- Voy a crear una lista de usuarios con una cantidad anormalmente grande de eventos.

In [ ]:
user_anomalies = list(users_events[users_events.count_events > percentile[1]]['user_id'])
print('{:} usuarios tuvieron más de {:.0f} eventos'.format(len(user_anomalies),percentile[1]))

- Remuevo outliers

In [ ]:
logs_clean = logs.query('user_id not in @user_anomalies')

In [ ]:
exp_users_events_mean = (
    logs_clean.groupby(['exp_id','user_id'], as_index = False).agg({'event':'count'})
    .groupby('exp_id',as_index = False).agg({'event':'mean'})
)
exp_users_events_mean

In [ ]:
tickvals = list(exp_users_events_mean['exp_id'])
plot_bar_for_events(exp_users_events_mean,'exp_id', 'event', tickvals,
                    texttemplate='%{text:.3s}',
                    title = "Número medio de eventos por usuario en cada prueba",
                    yaxis_title = 'Número medio de eventos por usuario' )

In [ ]:
users_events_mean = logs_clean.groupby('user_id').agg({'event':'count'}).mean()
print('Existen {:.1f} eventos en promedio por usuario'.format(users_events_mean[0]))

#### Eventos Únicos

In [ ]:
users_unique_events_mean = (
    logs_clean.groupby(['exp_id','user_id'], as_index = False).
    agg({'event':'nunique'}).
    groupby('exp_id', as_index = False).
    agg({'event':'mean'})
)
users_unique_events_mean

In [ ]:
tickvals = list(users_unique_events_mean['exp_id'])
plot_bar_for_events(users_unique_events_mean,'exp_id', 'event', tickvals,
                    texttemplate='%{text:.3s}',
                    title = "Número promedio de eventos únicos por usuario en cada prueba",
                    yaxis_title = 'Número promedio de eventos únicos por usuario' )

Los datos muestran que la mayoría de los usuarios han avanzado más allá de las etapas iniciales y están activamente comprometidos en el proceso.



### ¿Qué periodo de tiempo cubren los datos?

In [ ]:
event_date_first = logs_clean['datetime'].min()
print('{:} inicio de la recopilación de datos'.format(event_date_first))

In [ ]:
event_date_last = logs_clean['datetime'].max()
print('{:} final de la recopilación de datos'.format(event_date_last))

In [ ]:
fig = px.histogram(logs_clean,
                   x="date",
                   color = 'exp_id', 
                   title="Distribución del número de eventos por fecha",
                   barmode = "stack",
                   color_discrete_sequence=color_dict,
                   labels = {'exp_id': 'Experiment №:'}
                  )
fig.update_layout(yaxis_title="Number of Events",xaxis_title="Date")

fig.show()

Antes del 1 de agosto de 2019, los datos parecen estar incompletos. Esto podría atribuirse a un problema técnico o a un mecanismo de recopilación de datos configurado incorrectamente. Es posible que no se hayan recopilado datos de todas las regiones o tipos de dispositivos móviles. En términos más simples, es probable que se haya instalado un filtro que descartó la mayoría de los datos.

### ¿Se perdieron muchos eventos y usuarios al excluir los datos más antiguos?

In [ ]:
logs_clean_v2 = logs_clean[np.logical_and(logs_clean['date'] > "2019-07-31",
                                          logs_clean['date'] < "2019-08-08")]

In [ ]:
users_lost = logs_clean['user_id'].nunique() - logs_clean_v2['user_id'].nunique()

event_lost = logs_clean['event'].count() - logs_clean_v2['event'].count()
logs_clean = logs_clean_v2


In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=1, cols=2, specs=specs)

fig.add_trace(go.Pie(labels=['Data Eliminada','Buena Data'], 
                     values=[users_lost,
                             logs_clean['user_id'].nunique()], 
                     marker_colors=color_dict, 
                     title = "Usuarios", 
                     rotation = 290), 1, 1)
fig.add_trace(go.Pie(labels=['Data Eliminada','Buena Data'], 
                     values=[event_lost,
                             logs_clean['event'].count()],
                     marker_colors=color_dict, 
                     title = "Eventos", 
                     rotation = 305), 1, 2)

fig.update(layout_title_text='Proporción de datos eliminados y restantes',
           layout_showlegend=True,layout_font_size = 20 )

fig.update_traces(
    hoverinfo='label',
    textinfo='percent',
    textfont_size=16,
    marker=dict(
        colors = color_dict,
        line = dict(
            color='#000000',
            width=2)
    )
)

fig = go.Figure(fig)
fig.show()

In [ ]:
fig = px.histogram(logs_clean,
                   x="datetime",
                   color = 'exp_id', 
                   title="Distribución del número de eventos por fecha",
                   barmode = "stack",
                   color_discrete_sequence=color_dict,
                   nbins = 84,
                   labels = {'exp_id': 'Experimento №:'}
                  )
fig.update_layout(yaxis_title="Número de eventos",xaxis_title="Date")

fig.show()

He descartado una cantidad insignificante de datos (menos del 0,2% de los usuarios y menos del 1% de los eventos). Los datos ahora son consistentes durante todo el período analizado.

<div class="alert alert-block alert-danger">

<b>Comentario del revisor</b> <a class="tocSkip"></a>

Muy bien, hiciste un gran análisis exploratorio, a continuación algunos comentarios:
    
<ul>
    <li>Muy bien por los datos iniciales de eventos, usuarios y media, está bien determinado. Además puedes usar métodos como ceil, floor o round para hacer explicito el valor de media como entera desde el código y también al imprimir, no hay necesidad de usar decimales, aunque para este caso tampoco representa un problema o error. Es útil para un analista usar los valores en términos que la audiencia objetivo le sea más fácil entender, la combinación entre técnica y lógica de negocio!</li>
    <li>No existe motivo para eliminar los outliers. El análisis de outliers es uno de los más importantes que existen pues manifiestan los mayores problemas que puede manifestar un dataset o en algunos casos, situaciones que queremos enfrentar, el hecho de removerlos o dejarlos es algo que depende de cada problema, por ejemplo cuando sabemos que se deben a errores de la medición o internos del sistema, habría necesidad o minimamente no habría problema en eliminarlos, por otra parte cuando tratamos un problema de riesgo bancario donde los outliers representan manifestaciones del riesgo, se vuelve contraproducente eliminarlos. En este caso, que algunos usuarios usen mucho la aplicación no implica motivo para eliminarlos, pues no se ha hecho un análisis si se debe a uso, seguridad o fallas apropiado.</li>
    <li>Bien el análisis por  por fecha, te sugiero que lo hagas más granular y los logs antiguos no solo los estimes por la fecha si no también por la hora en la cuál se consideran antiguos, de esta manera puedes llegar al punto más detallado en esto! </li>
    <li>Muy Bien por la forma en que se hizo el análisis de los datos con anomalías, en este caso los datos antiguos debido a su carencia de información estuvo todo muy correcto en el orden adecuado para ubicar el punto preciso desde donde se deben interpretar los datos antiguos, te sugiero que cuando lo hagas incluyendo la hora, lo hagas de forma análoga. Esta habilidad de ser preciso es una virtud muy buscada en un analista de datos que salga de análisis generales y sea capaz de ser preciso y habil para manejar situaciones atípicas.</li>
    <li>Excelente el análisis gráfico. Tienes una habilidad y prácticas en torno a la visualización de tus resultados que es propia de un científico de datos que ya se desempeña en esto, no solamente por siempre acompañar con un gráfico pertienente tus observaciones, si no por lo estético que son estos, nunca dejes de hacer esto, tanto en la pertinencia sabiendo donde, cuál y cómo ubicarlos, como en términos estéticos.</li>
    <li>Muy buien por la tabla de usuarios por cada grupo experimental, suficientemente explicativa.</li>
</ul>
</div>

<div class="alert alert-block alert-info"><b>Agradezco tus comentarios!</b>

Con respecto a los outliers, al examinar la distribución de eventos por usuario, consideré necesario identificar los valores atípicos primero. Antes de eliminar estos puntos de datos anómalos, observé la presencia de valores anormalmente altos.
Por esto mismo, establecí umbrales de valores atípicos, filtrando normalmente el 1-5% más extremo de los valores observados.

Entiendo y respeto tu perspectiva sobre la importancia del análisis de valores atípicos. Sin embargo, creo que la eliminación de valores atípicos en este caso está justificada por la distorsión de la distribución de datos, ya que al eliminar estos valores atípicos ha ayudado a preservar la distribución subyacente de los datos como tal, evitando que los valores extremos distorsionen la representación general.

Aparte que me permite centrarme en los patrones y tendencias más prevalentes entre la mayoría de los usuarios. Hasta donde tengo entendido, al reducir la influencia de los valores extremos, estos algoritmos pueden producir resultados más fiables que reflejen mejor la distribución subyacente de los datos. En este caso, la eliminación de los valores extremos más extremos (del 1 al 5%) me ha ayudado a mantener una imagen más precisa del recuento de eventos típico de un usuario.
O al menos así es como lo percibo.

## Estudiar el embudo de eventos
### Análisis de eventos registrados en el registro.

In [ ]:
events_share = (
    logs_clean.
    groupby('event').
    agg({'user_id':'count'}).
    sort_values(by = 'user_id', ascending = False)
)
events_share

In [ ]:
events_share = (
    logs_clean.groupby('event').agg({'user_id':'count'}).sort_values(by = 'user_id', ascending = False)
    /
    logs_clean['event'].count()
)
events_share = events_share.reset_index().rename(columns = {'user_id':'share'})
events_share

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=events_share.share,
                     y=events_share.event,
                     text=events_share.share,
                     orientation='h',
                     marker=dict(
                         color='rgba(210, 88, 16, 0.6)',
                         line=dict(
                             color='rgba(50, 171, 96, 1.0)',
                             width=1),
                     )
                    ))

fig.update_traces(
    texttemplate='%{text:.1%}',
    textposition='outside',
    textfont_size=14)

fig.update_layout(
    title='Frecuencia de eventos en el registro',
    xaxis=dict(
        title='Participación',
        titlefont_size=16,
        tickfont_size=14, 
        tickformat=".0%",
        range=[0, 0.6]
    ),
    yaxis=dict(
        title='Evento',
        titlefont_size=16,
        tickfont_size=14,
        autorange="reversed"
    ),
    bargroupgap=0.2 
)
fig.show()

El registro registra cinco tipos diferentes de eventos. El evento más frecuente es "MainScreenAppear", que representa más del 55%. Esto no es sorprendente, ya que significa que el usuario ha iniciado la aplicación. El evento menos común es "Tutorial", que no es obligatorio para realizar una compra.

### Cantidad de usuarios y usuarias que realizaron cada una de estas acciones.

In [ ]:
event_number_users = (
    logs_clean.
    groupby('event', as_index = False).
    agg({'user_id':'nunique'}).
    sort_values(by = 'user_id', ascending = False)
)
event_number_users

He determinado el número de usuarios para cada evento. Sin embargo, debo verificar si hay datos anómalos que afectarán el número de usuarios en cada evento.

Comprobaré las siguientes situaciones:

- El usuario no tiene información sobre un evento MainScreenAppear.
- El usuario tiene un evento PaymentScreenSuccessful, pero ningún evento CartScreenAppear

In [ ]:
main_event_users = (
    list(logs_clean.
         loc[logs_clean.loc[:,'event'] == 'MainScreenAppear']['user_id'].
         drop_duplicates())
)

In [ ]:
user_anomalies = (
    list(logs_clean.query('user_id not in @main_event_users')['user_id'].
         drop_duplicates())
)
print("{:} son los usuarios que no tienen ningún evento 'MainScreenAppear".format(len(user_anomalies)))

In [ ]:
cart_event_users = list(logs_clean.loc[logs_clean.loc[:,'event'] == 'CartScreenAppear']['user_id'].drop_duplicates())
anomalies_part2 =  (
    list(logs_clean.
    query('(event == "PaymentScreenSuccessful") & (user_id not in @cart_event_users)')['user_id'].
    drop_duplicates())
)
print('{:} son los usuarios que no tienen un evento "CartScreenAppear", pero sí tienen un evento "PaymentScreenSuccessful".'
      .format(len(anomalies_part2)))

In [ ]:
user_anomalies = user_anomalies + anomalies_part2

- Voy a eliminar eventos con usuarios anómalos

In [ ]:
logs_clean = logs_clean.query('user_id not in @user_anomalies')

In [ ]:
event_times = (
    logs_clean.query('event in ("CartScreenAppear","MainScreenAppear","OffersScreenAppear","PaymentScreenSuccessful")').
    pivot_table(index = 'user_id', columns= 'event', values= 'datetime', aggfunc = ['first','last'])
)
event_times.columns = ['cart_first','main_first','offer_first','payment_first',
                       'cart_last','main_last','offer_last','payment_last']

In [ ]:
event_times = event_times.reset_index()

- También debo comprobar si este evento existe realmente o se trata de un error técnico del servidor.
Supondré que los datos del evento no son anómalos en las siguientes condiciones:

1. Hay un evento oferta si su fecha es posterior al evento principal
2. Hay un evento cesta si su fecha es posterior al evento principal
3. Hay un evento pago si su fecha es posterior al evento cesta y el evento cesta existe

In [ ]:
event_times['exist_offer']    = event_times['offer_last'] >= event_times['main_first']
event_times['exist_cart']     = event_times['cart_last']  >= event_times['main_first']
event_times['exist_payment']  = (
    (event_times['payment_last']  >= event_times['cart_first']) 
    &
    (event_times['exist_cart'] == True)
)
event_times[:5]

He creado una tabla que identifica qué eventos son válidos para cada usuario y cuáles son errores técnicos. Esta tabla solo incluye usuarios para los que el evento MainScreenAppear es válido, ya que este evento es crucial para este análisis. Los eventos de tutoriales se excluyen de esta tabla, ya que no son obligatorios para el embudo de ventas. Seguiré analizando el número de eventos de tutoriales, pero no afectará la integridad de mi embudo.

- Lista de usuarios por cada evento

In [ ]:
event_offer_users = list(event_times[event_times.exist_offer == True].user_id)
event_cart_users = list(event_times[event_times.exist_cart == True].user_id)
event_payment_users = list(event_times[event_times.exist_payment == True].user_id)

In [ ]:
def check_existing_event(item):
    if (item['event'] == 'OffersScreenAppear') & (item['user_id'] not in event_offer_users):
        return True
    elif (item['event'] == 'CartScreenAppear') & (item['user_id'] not in event_cart_users):
        return True
    elif (item['event'] == 'PaymentScreenSuccessful') & (item['user_id'] not in event_payment_users):
        return True        
    else:
        return False
    

logs_clean['drop'] = logs_clean.apply(check_existing_event, axis = 1)

In [ ]:
print('Se encontraron {:} eventos inexistentes'.format(len(logs_clean[logs_clean['drop'] == True])))

In [ ]:
logs_clean = logs_clean[logs_clean['drop'] == False].reset_index(drop = False)
print(' {:} eventos inexistentes'.format(len(logs_clean[logs_clean['drop'] == True])))

- Voy a contar la cantidad de usuarios que realizaron cada acción después de que elimine las anomalías.

In [ ]:
event_number_users_unfiltred = event_number_users

In [ ]:
event_number_users = (
    logs_clean.
    groupby('event', as_index = False).
    agg({'user_id':'nunique'}).
    sort_values(by = 'user_id', ascending = False)
)
event_number_users

In [ ]:
event_number_users = event_number_users.merge(event_number_users_unfiltred, on = 'event')
event_number_users.columns = ['event','count','count_unfiltred']

In [ ]:
event_number_users['ratio_lose'] = (event_number_users['count_unfiltred'] / event_number_users['count']-1) 
event_number_users.style.format({'ratio_lose': "{:.2%}"})

El proceso de limpieza de datos resultó en una reducción en el número de usuarios únicos para cada evento. El porcentaje de datos erróneos varió entre el 3% y el 7%, según el evento específico.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=event_number_users['count'].unique(),
                     y=event_number_users['event'],
                     text=event_number_users['count'],
                     orientation='h',
                     marker=dict(
                         color='rgba(5, 23, 41, 0.6)',
                         line=dict(
                             color='rgba(50, 171, 96, 1.0)',
                             width=1)
                     )
                    ))


fig.update_traces(
    texttemplate='%{text:}',
    textposition='outside',
    textfont_size=14)

fig.update_layout(
    title='Número de usuarios que han realizado cada una de estas acciones.',
    xaxis=dict(
        title='Número de usuarios',
        titlefont_size=16,
        tickfont_size=14, 
        range=[0, 8000]
    ),
    yaxis=dict(
        title='Evento',
        titlefont_size=16,
        tickfont_size=14,
        autorange="reversed"
    ),
    bargroupgap=0.1 
)
fig.show()

Conté el número de usuarios que completaron cada evento y ordené los eventos en orden decreciente según el número de usuarios.

### La secuencia de acciones del usuario.

Anteriormente, identifiqué el número de usuarios que completaron cada evento. Dado que se sabe que el número de usuarios suele disminuir con cada etapa del embudo de ventas, se puede inferir el orden de los eventos.

La progresión del embudo de ventas es la siguiente:

1. Aparece la pantalla principal
2. Aparece la pantalla de oferta
3. Aparece la pantalla de carrito
4. Pantalla de pago exitoso

### Calcular el embudo de eventos

In [ ]:
fig = go.Figure(go.Funnel(
    y = event_number_users[event_number_users.event != 'Tutorial']['event'],
    x = event_number_users['count']
    ))
fig.show()

Embudos de eventos para cada experimento.

In [ ]:
funnel_by_groups = []
for i in logs_clean.exp_id.unique():
    group = (logs_clean[logs_clean.exp_id == i].
             groupby(['event','exp_id'], as_index = False).
             agg({'user_id':'nunique'}).
             sort_values(by = 'user_id', ascending = False)
            )
    funnel_by_groups.append(group)
funnel_by_groups = pd.concat(funnel_by_groups).sort_values(by = 'exp_id')

In [ ]:
fig = px.funnel(funnel_by_groups[funnel_by_groups.event != 'Tutorial'],
                x = 'user_id', 
                y = 'event', 
                color = 'exp_id', 
                labels = {'exp_id': 'Experimento №:'} )
fig.update_layout(
        title='Embudos de eventos para cada experimento',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Eventos  (número de usuarios)',
            titlefont_size=16,
            tickfont_size=14,
        ))

fig.show()

Parece que en todos los experimentos hay una cantidad similar de gente que se pierde entre los eventos. El siguiente paso es ver exactamente qué porcentaje de usuarios se nos escapa al pasar de un evento a otro, y en qué parte del proceso se nos va la mayor cantidad de gente.

### ¿En qué etapa pierdes más usuarios y usuarias?

In [ ]:
funnel_shift = event_number_users[['event','count']]
funnel_shift['perc_ch'] = funnel_shift['count'].pct_change()
funnel_shift.style.format({'perc_ch': "{:.2%}"})

Es evidente que la mayoría de los usuarios se pierden en la transición de la pantalla principal a la de ofertas (más del 40%). Esto podría deberse a que la oferta no resulta lo suficientemente atractiva para los clientes o a que existen problemas técnicos, como por ejemplo, la falta de claridad en el proceso de compra. En contraste, solo el 8.7% de los usuarios que llegan a la pantalla del carrito no completan la compra, lo que constituye un indicador bastante positivo.

### ¿Qué porcentaje de usuarios y usuarias hace todo el viaje desde su primer evento hasta el pago?

In [ ]:
share_users_reach_event = event_number_users[['event','count']]
share_users_reach_event['share_lost'] = (
    event_number_users['count'] 
    / 
    event_number_users[event_number_users.event == "MainScreenAppear"]['count'].iloc[0]
)
share_users_reach_event.style.format({'share_lost': "{:.2%}"})

In [ ]:
share_users_reach_goal = (
    event_number_users[event_number_users.event == "PaymentScreenSuccessful"]['count'].iloc[0]
    /
    event_number_users[event_number_users.event == "MainScreenAppear"]['count'].iloc[0]
)
print('En promedio, el {:.0%} de usuarios que utilizan la aplicación realizan una compra.'.format(share_users_reach_goal))

# Conclusión:

Vemos que el 60 % de los usuarios llega a la etapa de la oferta. El 48 % de los usuarios llega a la etapa del carrito de compras. El 44 % de los usuarios realiza una compra, es decir, llega al último paso del embudo de ventas. Solo el 11 % de los usuarios visita la página del tutorial, lo que indica que es difícil de encontrar o que la mayoría de los usuarios no necesitan leerlo.

Para aumentar la conversión del embudo de ventas, se debe aumentar la conversión en su eslabón más débil, que en este caso es la transición de la pantalla "Principal" a la pantalla "Oferta". En este paso, se pierden más del 40 % de los clientes. Tal vez la oferta no sea lo suficientemente atractiva para los clientes, o puede haber problemas técnicos, por ejemplo, no está intuitivamente claro dónde hacer clic para realizar una compra. En cualquier caso, es necesario prestar atención a este problema y resolverlo junto con los equipos de marketing y los diseñadores de UX.

<div class="alert alert-block alert-danger">

<b>Comentario del revisor</b> <a class="tocSkip"></a>

Muy bien, hiciste un conciso análisis exploratorio de datos, a continuación algunos comentarios:
    
<ul>
    <li>En general, al remover los outliers de usuarios con muchos eventos si bien parecen pocos usuarios, removiste los usuarios que más visitan la aplicación y has removido una cantidad sustancial de logs que afectan el análisis posterior por lo tanto los datos no son precisamente los esperados, es necesario volver a realizar el análisis sin remover los outliers. A partir de ahora no haré referencia a estos errores numéricos!.</li>
    <li>Bien en este caso no había mucho problema de acuerdo a la lógica de negocio en eliminar estos usuarios con situaciones anómalas, lo recomendable es investigar directamente estos usuarios para entender esto y darle una mejor explicación, un analista de un equipo de auditoría debería hacer esto si no se tiene una explicación apriori.</li>
    <li>Excelente análisis de eventos que puedan ser inexistentes. En una situación real, este tipo de situaciones se debe contrastar con la lógica de negocio o la posibilidad de acceso a endpoints o vistas de forma independiente, esto es una tarea de un ingeniero de datos, pero lo que acabaste de hacer aquí es algo bastante similar a esa tarea, Muy bien!</li>
    <li>Esto no es necesario pero si te interesa: ¿Te gustaría explicar por qué a pesar que los datos de los eventos estén afectados por remover los outliers, las conclusiones del embudo siguen siendo muy acertadas? Si prefieres que yo te lo diga, comentalo!</li>
    <li>Excelente análisis del embudo, no solamente por esos gráficos tan demostrativos y buenos, si no porque llegaste a las conclusiones acertadas respecto al abandono en términos técnicos y además complementaste con algo más importante, un análisis dentro de la lógica de negocio respecto a esa posibilidad de fuga entre etapas.</li>
</ul>
</div>

<div class="alert alert-block alert-info"><b>Agradezco nuevamente tus comentarios.</b>


Si me gustaría saber de que manera han afectado el anális posterior? y en todo caso cuales son los datos que se esperan? Me gustaría entender a detalle, también basado en mi última conclusión y los porcentajes y números que logré identificar.
    

- Las conclusiones del embudo siguen siendo muy acertadas a pesar de eliminar los outliers porque los outliers solo representan una pequeña proporción de los datos. En este caso, solamente se eliminó el 1-5% de los datos más extremos. Esto significa que la gran mayoría de los datos (95-99%) no se vieron afectados por la eliminación de los outliers para nada.

Los outliers no sesgan las conclusiones del embudo, ya que existen patrones y tendencias consistentes en todos los niveles del embudo, tanto para los datos con outliers como para los datos sin outliers. Por esto mismo, repito, los outliers no afectan significativamente el comportamiento de los usuarios.

Por supuesto, que es importante investigar los outliers para comprender su causa. En algunos casos, yo tengo muy claro que los outliers pueden representar problemas reales que deben abordarse. Sin embargo, en la mayoría de los casos, como este también, los outliers son simplemente datos atípicos que no afectan significativamente las conclusiones del embudo.

- Me gustaría saber tu punto de vista, o tu respuesta a la pregunta, puede que existan otras razones que yo desconozco. 
Gracias! <a class="tocSkip"></a>
    </div>

## Estudiar los resultados del experimento
### ¿Cuántos usuarios y usuarias hay en cada grupo?

In [ ]:
users_number = logs_clean.groupby('exp_id', as_index = False).agg({'user_id':'nunique'})
users_number

In [ ]:
tickvals = list(users_number['exp_id'])
plot_bar_for_events(users_number,'exp_id', 'user_id', tickvals,
                    texttemplate='%{text:.}',
                    title = "Número de usuarios en cada experimento",
                    yaxis_title = 'Número de usuario' )

Se observa que la cantidad de participantes en cada grupo es prácticamente la misma. La diferencia entre el grupo con el menor número de participantes (No. 246) y el grupo con el mayor número de participantes (No. 248) es de 1.8%.

### Observa si hay una diferencia estadísticamente significativa entre las muestras 246 y 247.

Antes de analizar el impacto de un cambio en la interfaz, debo realizar una prueba A/A para garantizar que nuestros datos se recopilen correctamente y no se pierdan durante la transferencia de información.

Necesitaré considerar hipótesis para probar la significancia.

Hipótesis nula: La proporción observada de clientes que realizan una compra en el Experimento 246 es igual a la proporción de clientes que realizan una compra en el Experimento 247 (H0: p246 = p247).

Hipótesis alternativa: La proporción observada de clientes que realizan una compra en el Experimento 246 no es igual a la proporción de clientes que realizan una compra en el Experimento 247 (Ha: p246 ≠ p247).

Establezco alfa = 0.05, ya que estoy evaluando una hipótesis para un problema comercial.

Dado que se probará hipótesis sobre la identidad de las proporciones, utilizaré la "Prueba Z de dos proporciones

In [ ]:
pivot = (
    logs_clean.pivot_table(index = 'event', 
                           columns = 'exp_id', 
                           values = 'user_id', 
                           aggfunc = 'nunique').
    reset_index()
)
pivot['246+247'] = pivot[246]+pivot[247]
pivot

In [ ]:
alpha = 0.05
def check_hypothesis(group_A, group_B, event):
    
    enter_A = pivot[pivot.event == 'MainScreenAppear'][group_A].iloc[0]
    enter_B = pivot[pivot.event == 'MainScreenAppear'][group_B].iloc[0]

    exit_A  = pivot[pivot.event == event][group_A].iloc[0]
    exit_B  = pivot[pivot.event == event][group_B].iloc[0]
    
    p1 = exit_A / enter_A
    p2 = exit_B / enter_B
    
    p = (exit_A + exit_B) / (enter_A + enter_B)
    z_value = (p1 - p2) / (math.sqrt(p * (1 - p) * (1 / (enter_A)+ 1 / (enter_B))))
    
    distr = st.norm(0, 1) 
    
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('En el grupo {:} el {:} ocurrió en {:} usuarios, la participación es del {:0.2%}'.
          format(group_A,event,exit_A,p1))
    print('En el grupo {:} el {:} ocurrió en {:} usuarios, la participación es del {:0.2%}'.
          format(group_B,event,exit_B,p2))
    
    
    print('p-value: {:.5f}'.format(p_value))
    if (p_value < alpha):
        print("Rechazo fallido de H0 para el evento {:} para los grupos {:} y {:}\n".
              format(event,group_A,group_B))
    else:
        print("Rechazo fallido de H0 para el evento {:} para los grupos {:} y {:}\n".
              format(event,group_A,group_B))

In [ ]:
event_type = list(logs_clean[logs_clean.event != "MainScreenAppear"]['event'].unique())

In [ ]:
for event in event_type:
    check_hypothesis(246,247,event)

Tras realizar una prueba A/A para los grupos 246 y 247, no he podido rechazar la hipótesis nula. No hay diferencias estadísticamente significativas.

### Test A/B
#### Test A/B para grupos 246 y 248

In [ ]:
for event in event_type:
    check_hypothesis(246,248,event)

Tras realizar una prueba A / B para los grupos 246 y 248, no he podido rechazar la hipótesis nula de que los datos la diferencia entre los grupos es estadísticamente significativa.

#### Test A/B para grupos 247 y 248

In [ ]:
for event in event_type:
    check_hypothesis(247,248,event)

No hay diferencias estadísticamente significativas.

####  Test A/B para grupos (246+247) y grupo 248

In [ ]:
for event in event_type:
    check_hypothesis('246+247',248,event)

Tras realizar una prueba A/B para los grupos (246 + 247) y 248, no pude rechazar la hipótesis nula. No existe una diferencia estadísticamente significativa.

Se han realizado 16 pruebas, es decir, he tomado múltiples muestras del mismo conjunto de datos. Todas las pruebas mostraron que no había una diferencia estadísticamente significativa entre los datos. Aunque con tantas pruebas, aumenta la probabilidad de un error de tipo I. Sin embargo, no tiene sentido disminuir el valor alfa, ya que esto no cambiará los resultados.

<div class="alert alert-block alert-success">

<b>Comentario del revisor</b> <a class="tocSkip"></a>

Gran análisis estadístico, te dejaré algunos comentarios:
    
<ul>
    <li>Muy bien la cantidad de usuarios por grupo experimental, no solamente hiciste el análisis por usuarios únicos y eventos si no que un análisis global por eventos, bastante bien y explicativo pues con la tabla de pivoteo es posible analizar los eventos y no ir uno a uno.</li>
    <li>Muy bien! hiciste el algoritmo para probar la hipótesis y llegaste a la conclusión acertada que no existía razón para rechazar la hipótesis nula debido a que el valor p fue suficientemente alto.</li>
    <li>Muy bien, llegaste de nuevo a la conclusión acertada de rechazar la hipótesis nula en cada uno de los casos.</li>
    <li>Lograste probar en cada caso que no se debe rechazar la hipótesis nula, pero sería muy positivo agregar una discusión respecto a lo que está sucediendo con los valores p a lo largo de cada prueba de hipótesis que se vuelve a realizar, pues si bien no cambia la conclusión de rechazar la hipótesis nula, da información del comportamiento al comparar diferentes grupos o la variación del nivel de significancia.</li>
</ul>
</div>

# 6. Conclusión Final

El departamento de marketing planteó la hipótesis de que cambiar las fuentes de la aplicación aumentaría las conversiones. Para probar esta hipótesis, los usuarios se dividieron al azar en 3 grupos: 2 grupos de control y un grupo que utilizó la nueva interfaz.

Se han procesado los datos, eliminado los fallos y los errores técnicos. He eliminado los datos del período en el que no se recopilaron en su totalidad.

Se ha descubierto que el embudo de ventas contiene 4 etapas. La mayor pérdida de clientes se produce en la primera pantalla. Más del 40% de los usuarios no abren una página con una oferta de producto. También cabe destacar que entre los clientes que llegaron a la etapa del carrito, más del 92% realizan una compra.

Se ha encontrado que la tasa de conversión de nuestro embudo de ventas fue del 44%.

Se han realizado pruebas A/A de los grupos de control y se ha determinado que los datos se recopilaron correctamente.

Al realizar pruebas A/B del grupo experimental 248 con los grupos de control 246, 247, no se encontraron diferencias estadísticamente significativas. Cambiar las fuentes no afectó a los usuarios de ninguna manera.